In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
mapbox_access_token='pk.eyJ1IjoiYXFzYXNhZGFmIiwiYSI6ImNrc241ajRubzA3NjYydmswaXJ1bHlrYTMifQ.WHon85sWay7z-RpuQDKX1w'

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import missingno as msno
from urllib.request import urlopen
import json
import plotly.io as pio
import plotly.offline as pyo
pyo.init_notebook_mode()


# United States

In [ ]:
fig = go.Figure(go.Scattergeo())
fig.update_geos(
    visible=True, resolution=110, scope="usa",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="steelblue"
)
fig.update_layout(height=300, margin={"r":0,"t":0,"l":0,"b":0},paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',)
fig.show()

# About Data:

**City:** The name of the city/town.

**County Fips:** The 5-digit FIPS code for the primary county. The first two digits correspond to the state's FIPS code.

**Lat:** The latitude of the city/town.

**Lng:** The longitude of the city/town.

**Population:** An estimate of the city's urban population.

**Density:** The estimated population per square kilometer.

**Source:** For some cities, our data is generated from a polygon representing the city, for others we simply have a point.

**Military:** TRUE if this place is a military establishment such as a fort or base.

**Incorporated:** TRUE if the place is a city/town. FALSE if the place is just a commonly known name for a populated area.

**Ranking:** An integer from 1-5 that captures the importance of a city (1 is most important, 5 least important).

**Timezone:** The city's time zone in the tz database format. (e.g. America/Los_Angeles)


In [ ]:
df = pd.read_csv('/kaggle/input/us-cities/uscities.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df['county_fips'] = df['county_fips'].apply(lambda x:str(x) if x>=10000 else '0'+str(x))

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

# Population:

In [ ]:
fig = px.scatter_mapbox(df, lat="lat", lon="lng", hover_name="city", hover_data=["state_name", "population"], 
                        color="population",
                        size="population", color_continuous_scale=px.colors.sequential.Rainbow, size_max=40,
                        zoom=3, height=600, mapbox_style="open-street-map",width=1000,
                       title='City Population')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',)
fig.show()

# Density:

In [ ]:
fig = px.scatter_mapbox(df, lat="lat", lon="lng", hover_name="city", hover_data=["state_name", "density"], 
                        color="density",
                        size="density", color_continuous_scale=px.colors.sequential.Sunset, size_max=40,
                        zoom=3, height=600,width=1000, mapbox_style="open-street-map",
                       )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',)
fig.show()

# Ranking Of Cities Of US:

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=df.county_fips, z=df.ranking,
                                    colorscale="fall", zmin=1, zmax=3, marker_line_width=0))
fig.update_layout(mapbox_style="light", mapbox_accesstoken=mapbox_access_token,width=1000,
                  mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',)
fig.show()

# Military:

**Military base across US Cities**

In [ ]:
militry_base = df[df['military'] == True]

fig = px.scatter_mapbox(militry_base, lat="lat", lon="lng", hover_name="city", hover_data=["state_name", "military"],
                        color_discrete_sequence=["gold"], zoom=4, height=700,width=1000,)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=mapbox_access_token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},paper_bgcolor='rgb(248, 248, 255)',
     plot_bgcolor='rgb(248, 248, 255)',)
fig.show()